1. import modules

In [1]:
import pandas as pd
import datetime
import numpy as np

2. Read the file

In [ ]:
def read_file(path):
    df = pd.read_excel(path, header = None)

In [ ]:
df = read_file("/Users/logan/Downloads/NZ_Admin_JOBS.xlsx")

3. Set a variable

In [ ]:
dict_index = {'Admin':5}
location_index = 9

4. define method

In [ ]:
def deduplication(x):
    mid = len(x)//2
    if x!='unknown':
        return x[:mid+1]
    else:
        return 'unknown'

In [ ]:
def apply_salary(x):
    if 'classification:' in x:
        return 'unknown'
    else:
        return x

In [ ]:

def apply_classification(x):
    if 'classification' in x:
        return x[15:]
    else:
        return 'unknown'

In [ ]:
def apply_posted_time(x):
    if 'd' in x:
        return -int(x[:-1])
    if 'm' in x:
        return -(int(x[:-1])*30)
    else:
        return 0

5. set clean_area

In [ ]:
def clean_area(df, index):
    ##split 'location','area'
    df[['location','area']] = df[3].str.split('area:', expand=True)
    df['location'] = df['location'].apply(lambda x:x[location_index:])
    df['area'].fillna(value='unknown',inplace = True)
    df['area'] = df['area'].apply(deduplication)
    df['location'].fillna(value='unknown',inplace = True)
    df['location'] = df['location'].apply(deduplication)
    
    ##split 'subclassification','classification'
    df[['subclassification','classification']]=df[5].str.split('subClassification:', expand = True)
    df['classification'].fillna('unknown', inplace = True)
    df['classification'] = df['classification'].apply(deduplication)
    
    ##split 'posted time','featurned at'
    df[['posted time','featurned at']] = df[4].str.split(',', expand = True)[[0,2]]
    df['featurned at'].fillna('unknown', inplace = True)
    
    ##clean 'subclassification','salary'
    df['salary'] = df['subclassification'].apply(apply_salary)
    df['subclassification']=df['subclassification'].apply(apply_classification)
    df[['classification1','classification2','classification3']] = df['subclassification'].str.split('&', expand = True).fillna('unknown')
    
    ##rename other columns
    df.rename(columns ={0:'Job title',1:'website',2:'company name'}, inplace = True )
    df['company name'].fillna(df['featurned at'], inplace = True)
    
    ##clean posted time
    df['posted time']=df['posted time'].str.extract('(\d+[a-z])')
    df['posted time'].fillna('unknown', inplace = True)
    df['posted time']=df['posted time'].astype('string')
    df['posted time']=df['posted time'].apply(apply_posted_time)
    
    ##clean salary
    df['salary']=df[df['salary'].str.contains('\d', na=False)]['salary'].str.replace(' to ','-',regex=True)
    df['salary']=df[df['salary'].str.contains('\d', na=False)]['salary'].str.replace('\d\%','',regex=True)
    df['salary']=df[df['salary'].str.contains('\d', na=False)]['salary'].str.replace(',','',regex=True)
    df['salary'].fillna('unknown' ,inplace = True)
    df['salary']=df[df['salary'].str.contains('\d', na=False)]['salary'].str.replace('5k','5000',regex=True)
    df['salary']=df[df['salary'].str.contains('\d', na=False)]['salary'].str.replace('0k','0000',regex=True)
    df['salary'].fillna('unknown', inplace=True)
    df[['low_salary','high_salary','place_holder']] = df['salary'].str.split('-', expand = True).fillna('unknown')
    df['low_salary']=df['low_salary'].str.extract('(\d+|\d+\.\d+|\d+ \d+)')
    df['high_salary']=df['high_salary'].str.extract('(\d+|\d+\.\d+|\d+ \d+)')
    df['low_salary']=df['low_salary'].fillna('unknown')
    df['high_salary']=df['high_salary'].fillna('unknown')
    df.loc[df['low_salary']=='unknown', 'low_salary'] = df['high_salary']
    df.loc[df['high_salary']== 'unknown','high_salary']= df['low_salary']
    
    ##set random salary
    for r, row in enumerate(df['low_salary'].values):
    if row == 'unknown':
        df['low_salary'][r] = np.random.randint(45000,75000,size=1)[0]
    
    for r, row in enumerate(df['high_salary'].values):
    if row == 'unknown':
        df['high_salary'][r] = max(np.random.randint(45000,75000,size=1)[0],df['low_salary'][r])
        
    df['low_salary']=df['low_salary'].astype('int32')
    df['high_salary']= df['high_salary'].astype('int32')
    df.loc[df['low_salary']<50, 'low_salary'] = df['low_salary']*8*200
    df.loc[df['high_salary']<50, 'high_salary'] = df['high_salary']*8*200
    
    return df

6. perform actions

In [ ]:
def clean_jobs(job_name):
    df = clean_area(df, dict_index['Admin'])
    return df

In [ ]:
df = clean_jobs('Admin')